In [3]:
import aiohttp
import asyncio
import requests
from config import client_id
from SPAC_Arbitrage import listOfWar
from requests.adapters import HTTPAdapter

In [4]:
class Node:
    def __init__(self, symbolW=None, symbolI=None, askW=None, bidW=None, exs=None, askI=None, bidI=None, percentUorD=None, upDown=None):
        self.symbolW = symbolW
        self.symbolI = symbolI
        self.askW = askW
        self.bidW = bidW
        self.exs = exs
        self.askI = askI
        self.bidI = bidI
        self.percentUorD = percentUorD
        self.upDown = upDown

In [5]:
list_of_data = []

newWar = []

errorCnt = 0

for i in listOfWar:
    if not (i.exsPrice == 0 or i.twentyAvg == 0 or i.close == 0 or i.shell):
        newWar.append(i)

print("Length: ", len(newWar))

Length:  219


In [6]:
def inTheMoney(bidI, exs):
    if (bidI - exs) > 0:
        return True
    else:
        return False
    
def upOrDownSide(bidI, exs, askW):
    if (inTheMoney(bidI, exs)):
        #print("HERE")
        return bidI - askW - exs - 0.65
    else:
        return 0

def percentUorD(bidI, exs, askW):
    if (inTheMoney(bidI, exs)):
        #print("ITM")
        value = upOrDownSide(bidI, exs, askW)
        #print(type(value))
        sumOf = askW + exs + 0.65
        return value / sumOf
    else:
        #print("Not ITM")
        return 0

def printThis(final_data):
    count = 1
    print("Length: ", len(final_data))
    for i in final_data:
        print("Warrant #", str(count), ": ", i.symbolW)
        print("Issuer: ", i.symbolI)
        print("Exercise Price: ", i.exs)
        print("Warrant Ask Price: ", i.askW)
        print("Warrant Bid Price: ", i.bidW)
        print("Issuer Ask Price: ", i.askI)
        print("Issuer Bid Price: ", i.bidI)
        print("Percent Upside or Downside: ", i.percentUorD)
        count+=1

        
def printUp(final_data):
    count = 1
    print("Length: ", len(final_data))
    for i in final_data:
        #print(count)
        if(i.percentUorD > 0):
            #print("HERE")
            print("Warrant: ", i.symbolW)
            print("Issuer: ", i.symbolI)
            print("Exercise Price: ", i.exs)
            print("Warrant Ask Price: ", i.askW)
            print("Warrant Bid Price: ", i.bidW)
            print("Issuer Ask Price: ", i.askI)
            print("Issuer Bid Price: ", i.bidI)
            print("Upside Money: $" + str(i.upDown))
            print("Percent Upside or Downside: " + str(i.percentUorD*100) + "%")
        count+=1


In [7]:
async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for war in newWar:
            task = asyncio.ensure_future(get_war_data(session, war))
            tasks.append(task)
            
        list_of_data = await asyncio.gather(*tasks)
        
        final_data = []
        
        for i in list_of_data:
            if not (i.askW == 0):
                final_data.append(i)
    
    printUp(final_data)
            
async def get_war_data(session, war):
    #The daily prices endpoints
    #Exs, Ask, Bid
    symbolW = war.symbolW
    symbolI = war.symbolI
    exs = war.exsPrice
    
    quotes = symbolW + "," + symbolI
    
    #define payload
    
    payload = {
        'apikey': client_id,
        'symbol': quotes
    }
    
    #define endpt
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/quotes"
        #make request
    try:
        async with session.get(url = endpoint, params = payload, timeout = 2) as content:
            #convert to dic
            data = await content.json()


            askW = data[symbolW]['askPrice']
            bidW = data[symbolW]['bidPrice']
            askI = data[symbolI]['askPrice']
            bidI = data[symbolI]['bidPrice']
            percent = percentUorD(bidI, exs, askW)
            upDown = upOrDownSide(bidI, exs, askW)
            node = Node(symbolW, symbolI, askW, bidW, exs, askI, bidI, percent, upDown)
            return node
    except:
            askW = 0
            bidW = 0
            askI = 0
            bidI = 0
            node = Node(symbolW, symbolI, askW, bidW, exs, askI, bidI)

            return node
   
await main()

Length:  123
Warrant:  BRMK+
Issuer:  BRMK
Exercise Price:  2.88
Warrant Ask Price:  0.0892
Warrant Bid Price:  0.0712
Issuer Ask Price:  7.56
Issuer Bid Price:  7.0
Upside Money: $3.3808000000000002
Percent Upside or Downside: 93.41290893015032%
Warrant:  FRGE+
Issuer:  FRGE
Exercise Price:  11.5
Warrant Ask Price:  1.72
Warrant Bid Price:  1.04
Issuer Ask Price:  18.0
Issuer Bid Price:  17.72
Upside Money: $3.8499999999999983
Percent Upside or Downside: 27.757750540735387%
